In [2]:
import pandas as pd
import numpy as np
import re
import os
import datetime as dt

In [3]:
df_regal = pd.read_excel("General Claim Returns 51-motors.xlsx")
df_supp  = pd.read_excel("Regal_shipment_number.xlsx")

In [4]:
def rep(x):
    if x.startswith('51-102497-'):
        return '51-101920-00'
    elif x.startswith('51-103823-'):
        return '51-101984-00'
    elif x.startswith('51-104337-'):
        return '51-101879-00'
    elif x.startswith('51-104338-'):
        return '51-101880-00'
    elif x.startswith('51-104339-'):
        return '51-101920-00'
    elif x.startswith('51-104340-'):
        return '51-101984-00'
    else:
        return x
    
df_new = df_regal.copy() 
df_regal['Entered Failed Part'] = df_regal['Entered Failed Part'].apply(rep)
df_regal[df_regal['Entered Failed Part'].isin(['51-101920-00','51-101984-00','51-101879-00','51-101880-00','51-101920-00','51-101984-00'])]['Entered Failed Part'].value_counts()

51-101920-00    20537
51-101880-00    19310
51-101879-00     4239
51-101984-00     2553
Name: Entered Failed Part, dtype: int64

In [5]:
df_regal['part_family']= df_regal['Entered Failed Part'].apply(lambda x: "-".join(x.split('-',2)[:2]))
df_regal['part_family'] = df_regal['part_family'].replace({'51-102497':'51-101920' , '51-103823': '51-101984',\
                                                      '51-104337':'51-101879','51-104338':'51-101880',\
                                                        '51-104339':'51-101920','51-104340':'51-101984'})

df_regal[df_regal.part_family.isin(['51-101920','51-101984','51-101879','51-101880'])]['part_family'].value_counts()

51-101880    32694
51-101920    25933
51-101879     7095
51-101984     3488
Name: part_family, dtype: int64

In [7]:
df_regal["num_days"] = (pd.to_datetime(df_regal['Claim Transaction Date'])-pd.to_datetime(df_regal['Failed Unit Production Date'])).dt.days

In [8]:
df_regal.columns

Index(['Claim Number', 'Failed Unit Product Class',
       'Failed Unit Customer Name', 'Installer Name',
       'Entered Failed Unit Number', 'Entered Failed Unit Serial',
       'Entered Repl Unit', 'Entered Repl Unit Serial', 'Entered Failed Part',
       'Entered Failed Part Serial Number', 'Entered Repl Part',
       'Claim Transaction Date', 'TRANSACTION YEAR',
       'Failed Unit Production Date', 'Claim Type Name', 'QA Reason Code Name',
       'QA Reason Code Comments', 'Price', 'Labor Amount', 'Reserve Cost',
       'Failed Part Product Name', 'DIFFERENCE', 'part_family', 'num_days'],
      dtype='object')

In [9]:
df_new = df_regal.copy()
df_reg_filt = df_new[df_new['DIFFERENCE']=="KEEP"]
df_reg_filt.to_excel("FILTERED_REGAL.xlsx",index=False)

In [10]:
df_reg_filt['TRANSACTION YEAR'].sort_values().unique()

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
      dtype=int64)

In [11]:
df_reg_filt['num_claims'] = 1
df_reg_filt['Production_year'] = pd.to_datetime(df_reg_filt['Failed Unit Production Date']).dt.year


C:\Users\aamir.rafi\AppData\Local\Temp\ipykernel_9468\2323969533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reg_filt['num_claims'] = 1
C:\Users\aamir.rafi\AppData\Local\Temp\ipykernel_9468\2323969533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reg_filt['Production_year'] = pd.to_datetime(df_reg_filt['Failed Unit Production Date']).dt.year


In [12]:
df_reg_filt.columns

Index(['Claim Number', 'Failed Unit Product Class',
       'Failed Unit Customer Name', 'Installer Name',
       'Entered Failed Unit Number', 'Entered Failed Unit Serial',
       'Entered Repl Unit', 'Entered Repl Unit Serial', 'Entered Failed Part',
       'Entered Failed Part Serial Number', 'Entered Repl Part',
       'Claim Transaction Date', 'TRANSACTION YEAR',
       'Failed Unit Production Date', 'Claim Type Name', 'QA Reason Code Name',
       'QA Reason Code Comments', 'Price', 'Labor Amount', 'Reserve Cost',
       'Failed Part Product Name', 'DIFFERENCE', 'part_family', 'num_days',
       'num_claims', 'Production_year'],
      dtype='object')

In [14]:
df_reg_grp = df_reg_filt.groupby(['Failed Unit Product Class','TRANSACTION YEAR','Entered Failed Part'])[['num_claims','Reserve Cost']].sum().reset_index()
df_reg_grp = df_reg_grp.rename(columns={'TRANSACTION YEAR':'Year'})
df_reg_grp

,Failed Unit Product Class,Year,Entered Failed Part,num_claims,Reserve Cost
0,ACCESSORIES,2015,51-104359-00,1,70.22275
1,ACCESSORIES,2019,51-42537-01,1,124.19825
2,ACCESSORIES,2022,51-100998-19,1,60.83990
3,AIR HANDLER/ELEC,2015,51-101728-02,1,34.20425
4,AIR HANDLER/ELEC,2015,51-101728-03,1,39.20625
...,...,...,...,...,...
6172,REMOTE HEAT PUMP,2024,51-108350-01,4,196.25100
6173,REMOTE HEAT PUMP,2024,51-108351-01,2,82.45050
6174,REMOTE HEAT PUMP,2024,51-108504-01,1,57.28690
6175,REMOTE HEAT PUMP,2024,51-108708-06,1,41.01625


In [15]:
df_supp
# Unpivot the DataFrame
df_unpivoted = pd.melt(df_supp, id_vars=['Part Number'], var_name='Year', value_name='Value')
 # Drop rows where 'Value' is None 
df_unpivoted = df_unpivoted.dropna(subset=['Value']) 
df_unpivoted =df_unpivoted.rename(columns= {'Part Number':'Entered Failed Part','Value':'shipped_units'})
# df_unpivoted['Value'] = df_unpivoted['Value'].round(decimals=0).astype(object)
df_unpivoted

,Entered Failed Part,Year,shipped_units
0,51-100837-01,2015,25.0
1,51-100837-02,2015,25.0
2,51-100837-04,2015,6904.0
3,51-100837-05,2015,99.0
4,51-100837-07,2015,315.0
...,...,...,...
2754,51-42534-39,2023,250.0
2755,51-42534-40,2023,742.0
2756,51-42534-41,2023,269.0
2757,51-42534-42,2023,15.0


In [16]:
df_reg_grp[df_reg_grp['Entered Failed Part']=='51-100998-24']

,Failed Unit Product Class,Year,Entered Failed Part,num_claims,Reserve Cost
2656,PKG-COOLING,2016,51-100998-24,2,72.09850
2694,PKG-COOLING,2017,51-100998-24,7,277.10875
2736,PKG-COOLING,2018,51-100998-24,13,526.36825
2793,PKG-COOLING,2019,51-100998-24,20,670.35320
2837,PKG-COOLING,2020,51-100998-24,43,1566.85210
2894,PKG-COOLING,2021,51-100998-24,69,2419.01825
2953,PKG-COOLING,2022,51-100998-24,87,3642.64010
3019,PKG-COOLING,2023,51-100998-24,110,4778.96265
3093,PKG-COOLING,2024,51-100998-24,9,388.23840
3139,PKG-GAS/ELECT,2015,51-100998-24,1,36.04925


In [18]:
df_final_grped = df_unpivoted.merge(df_reg_grp,on=['Year','Entered Failed Part'],how='left')
df_final_grped[df_final_grped['Entered Failed Part']=='51-100998-24']
df_final_grped.to_excel('Final_data.xlsx',index=False)
# df_final_grped.head()

In [19]:
df_final_grped['Rolling_Avg_Shipped_Units'] = df_final_grped.groupby('Entered Failed Part')['shipped_units'].rolling(window=6, min_periods=1).mean().reset_index(level=0, drop=True) 
# Display the DataFrame with rolling average 
df_final_grped[['Entered Failed Part', 'Year', 'shipped_units', 'num_claims', 'Rolling_Avg_Shipped_Units']]
df_final_grped['Rolling_Avg_Shipped_Units']  = df_final_grped['Rolling_Avg_Shipped_Units'].round(decimals=0)
df_final_grped['Reserve Cost']  = df_final_grped['Reserve Cost'].round(decimals=2)

df_final_grped[df_final_grped['Entered Failed Part']=='51-100998-24']


,Entered Failed Part,Year,shipped_units,Failed Unit Product Class,num_claims,Reserve Cost,Rolling_Avg_Shipped_Units
25,51-100998-24,2015,6725.0,PKG-GAS/ELECT,1.0,36.05,6725.0
26,51-100998-24,2015,6725.0,PKG-HEAT PUMP,2.0,72.10,6725.0
242,51-100998-24,2016,32473.0,PKG-COOLING,2.0,72.10,15308.0
243,51-100998-24,2016,32473.0,PKG-GAS/ELECT,6.0,216.30,19599.0
244,51-100998-24,2016,32473.0,PKG-HEAT PUMP,10.0,360.49,22174.0
495,51-100998-24,2017,39948.0,PKG-COOLING,7.0,277.11,25136.0
496,51-100998-24,2017,39948.0,PKG-GAS/ELECT,11.0,311.75,30673.0
497,51-100998-24,2017,39948.0,PKG-HEAT PUMP,17.0,481.80,36210.0
795,51-100998-24,2018,43524.0,PKG-COOLING,13.0,526.37,38052.0
796,51-100998-24,2018,43524.0,PKG-GAS/ELECT,39.0,1207.54,39894.0


In [21]:
df_final_grped['rate'] = round((df_final_grped['num_claims']/df_final_grped['Rolling_Avg_Shipped_Units'])*100,2)
df_final_grped

,Entered Failed Part,Year,shipped_units,Failed Unit Product Class,num_claims,Reserve Cost,Rolling_Avg_Shipped_Units,rate
0,51-100837-01,2015,25.0,NaN,NaN,NaN,25.0,NaN
1,51-100837-02,2015,25.0,NaN,NaN,NaN,25.0,NaN
2,51-100837-04,2015,6904.0,PKG-GAS/ELECT,3.0,140.56,6904.0,0.04
3,51-100837-05,2015,99.0,NaN,NaN,NaN,99.0,NaN
4,51-100837-07,2015,315.0,NaN,NaN,NaN,315.0,NaN
...,...,...,...,...,...,...,...,...
2536,51-42534-40,2023,742.0,PKG-GAS/ELECT,13.0,1049.20,424.0,3.07
2537,51-42534-41,2023,269.0,PKG-COOLING,3.0,259.14,134.0,2.24
2538,51-42534-41,2023,269.0,PKG-GAS/ELECT,1.0,85.65,156.0,0.64
2539,51-42534-42,2023,15.0,NaN,NaN,NaN,6.0,NaN


In [22]:
df_final_grped.to_excel("SCORE_CARD_REGAL.xlsx",index=False)

### BY PRODUCT FAMILY

In [24]:
df_regal.columns

Index(['Claim Number', 'Failed Unit Product Class',
       'Failed Unit Customer Name', 'Installer Name',
       'Entered Failed Unit Number', 'Entered Failed Unit Serial',
       'Entered Repl Unit', 'Entered Repl Unit Serial', 'Entered Failed Part',
       'Entered Failed Part Serial Number', 'Entered Repl Part',
       'Claim Transaction Date', 'TRANSACTION YEAR',
       'Failed Unit Production Date', 'Claim Type Name', 'QA Reason Code Name',
       'QA Reason Code Comments', 'Price', 'Labor Amount', 'Reserve Cost',
       'Failed Part Product Name', 'DIFFERENCE', 'part_family', 'num_days',
       'Claims_count'],
      dtype='object')

In [25]:
# df_regal[df_regal['TRANSACTION YEAR']==2023].groupby('part_family')
df_regal['Claims_count'] =1
df_2023_regal = df_regal[df_regal['TRANSACTION YEAR']==2023].groupby(['Failed Unit Product Class','TRANSACTION YEAR','part_family'])[['Claims_count','Reserve Cost']].sum().reset_index()
df_2023_regal = df_2023_regal.rename(columns={'Reserve Cost':'Reserve_cost_2023','TRANSACTION YEAR':'Year'})
df_2023_regal.head()

,Failed Unit Product Class,Year,part_family,Claims_count,Reserve_cost_2023
0,AIR HANDLER/ELEC,2023,51-100837,1,58.41550
1,AIR HANDLER/ELEC,2023,51-101728,1720,99024.02150
2,AIR HANDLER/ELEC,2023,51-101774,13,555.10400
3,AIR HANDLER/ELEC,2023,51-101879,821,57610.80820
4,AIR HANDLER/ELEC,2023,51-101880,1652,124305.28935


In [26]:
df_reg_grp_fam = df_reg_filt.groupby(['Failed Unit Product Class','TRANSACTION YEAR','part_family'])[['num_claims','Reserve Cost']].sum().reset_index()
df_reg_grp_fam = df_reg_grp_fam.rename(columns={'TRANSACTION YEAR':'Year'})
df_supp
df_reg_grp_fam
# Unpivot the DataFrame
df_unpivoted_fam = pd.melt(df_supp, id_vars=['Part Number'], var_name='Year', value_name='Value')
 # Drop rows where 'Value' is None 
df_unpivoted_fam = df_unpivoted_fam.dropna(subset=['Value']) 
df_unpivoted_fam =df_unpivoted_fam.rename(columns= {'Part Number':'Entered Failed Part','Value':'shipped_units'})
df_unpivoted_fam['part_family'] =df_unpivoted_fam['Entered Failed Part'].apply(lambda x: "-".join(x.split('-',2)[:2]))
df_unpivoted_family = df_unpivoted_fam.groupby(['part_family','Year'])['shipped_units'].sum().reset_index()
df_final_grped_family = df_unpivoted_family.merge(df_reg_grp_fam,on=['Year','part_family'],how='left')
# df_final_grped_family[df_final_grped_family['Entered Failed Part']=='51-100998-24']
df_final_grped_family.to_excel('Final_data_family.xlsx',index=False)
df_final_grped_family['Rolling_Avg_Shipped_Units'] = df_final_grped_family.groupby('part_family')['shipped_units'].rolling(window=6, min_periods=1).mean().reset_index(level=0, drop=True) 
# Display the DataFrame with rolling average 
df_final_grped_family[['part_family', 'Year', 'shipped_units', 'num_claims', 'Rolling_Avg_Shipped_Units']]
df_final_grped_family['Rolling_Avg_Shipped_Units']  = df_final_grped_family['Rolling_Avg_Shipped_Units'].round(decimals=0)
df_final_grped_family['Reserve Cost']  = df_final_grped_family['Reserve Cost'].round(decimals=2)
df_final_grped_family['rate'] = round((df_final_grped_family['num_claims']/df_final_grped_family['Rolling_Avg_Shipped_Units'])*100,2)
# df_final_grped['rate'] = round((df_final_grped['num_claims']/df_final_grped['Rolling_Avg_Shipped_Units'])*100,2)
df_final_grped_family.to_excel("SCORE_CARD_REGAL_FAMILY.xlsx",index=False)

df_final_grped_family[df_final_grped_family['part_family'].str.startswith('51-100998')]

# df_unpivoted
# df_reg_grp.head()


,part_family,Year,shipped_units,Failed Unit Product Class,num_claims,Reserve Cost,Rolling_Avg_Shipped_Units,rate
28,51-100998,2015,39454.0,GAS,1.0,36.89,39454.0,0.00
29,51-100998,2015,39454.0,PKG-COOLING,12.0,406.45,39454.0,0.03
30,51-100998,2015,39454.0,PKG-GAS/ELECT,35.0,1256.19,39454.0,0.09
31,51-100998,2015,39454.0,PKG-HEAT PUMP,13.0,530.16,39454.0,0.03
32,51-100998,2015,39454.0,REMOTE CONDENSER,22.0,770.65,39454.0,0.06
33,51-100998,2016,66706.0,PKG-COOLING,35.0,1127.91,43996.0,0.08
34,51-100998,2016,66706.0,PKG-GAS/ELECT,148.0,5635.71,48538.0,0.30
35,51-100998,2016,66706.0,PKG-HEAT PUMP,80.0,2860.48,53080.0,0.15
36,51-100998,2016,66706.0,REMOTE CONDENSER,66.0,2040.44,57622.0,0.11
37,51-100998,2017,73399.0,AIR HANDLER/ELEC,1.0,32.60,63280.0,0.00


In [27]:
df_final_grped_family.merge(df_2023_regal,on=['Failed Unit Product Class','part_family','Year'],how='left').to_excel('SCORE_CARD_REGAL_FAMILY_UPDATED.xlsx',index=False)


In [28]:
##LET US DO THE COMMIT
if 'there'==2:
    print( True)
else:
    print(False)

SyntaxError: 'return' outside function (118140220.py, line 3)